# Setup

In [1]:
# my virtual environments are rarely properly connected to jupyter so this fixes that. 
# you prolly won't need this cell but running it won't hurt anything either
import sys
import os
current_dir = os.getcwd()  # Get the current working directory
venv_dir = os.path.join(current_dir, 'venv') 
python_version = str(sys.version_info.major) + '.' + str(sys.version_info.minor)
site_packages_path = os.path.join(venv_dir, 'lib', 'python' + python_version, 'site-packages')
sys.path.append(site_packages_path) 

# Instantiate a brand new model

In [2]:
# config file
from config import ModelConfig, TrainConfig
cfg = ModelConfig()
tcfg = TrainConfig()
print(cfg, '\n\n', tcfg)

# import the tokenizer specified by cfg
from tools import import_from_nested_path
imported_objects = import_from_nested_path(['tokenizers', cfg.tokenizer], 'tokenizer', ['get_tokenizer'])
get_tokenizer = imported_objects.get('get_tokenizer')
tokenizer = get_tokenizer(size = cfg.vocab_len)

# model modules
from modules.model import Model
model = Model(cfg).to(cfg.device)

import torch
#model = torch.compile(model) 
# ^if this takes too long & you're debugging you can comment it out, but def leave it on for full training runs if you've got CUDA

# print the number of parameters in the model
print(f'\n{sum(p.numel() for p in model.parameters())/1e3}K parameters\n')
print(model)

ModelConfig(dim=32, device='cpu', out_weight_share=True, linear_bias=False, tokenizer='bpe_v2_tinyStories', vocab_len=512, num_layers=2, second_resid_norm=False, mlp_hidden_mult=4, mlp_nonlinearity='SiLU', mlp_gated=True, num_q_heads=2, num_kv_heads=1, head_dim=16, theta=10000, max_seq_len=64, scale_first_resid=True, norm_type='RMSNorm', norm_affine=True, norm_bias=True, eps=1e-06) 

 TrainConfig(weight_decay=0.05, batch_size=32, max_iters=10, eval_interval=2, eval_samples=1, checkpoint_interval=None, beta1=0.9, beta2=0.95, lr_init=1e-06, lr_max=0.1, lr_min=0.001, warmup_iters=0, final_flat_iters=1, anneal_type='cos', num_restarts=3, T_mult=2)

43.264K parameters

Model(
  (token_embedder): Embedding(640, 32)
  (layers): ModuleList(
    (0-1): 2 x Layer(
      (pre_attn_norm): Norm()
      (attn): MQA(
        (Wq): Linear(in_features=32, out_features=32, bias=False)
        (Wk): Linear(in_features=32, out_features=16, bias=False)
        (Wv): Linear(in_features=32, out_features=16, 

# Training

In [3]:
from tools import get_data_loader
from train import scheduler_lambda, train

optimizer = torch.optim.AdamW(model.parameters(), lr = tcfg.lr_max, weight_decay = tcfg.weight_decay, betas = (tcfg.beta1, tcfg.beta2))
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=scheduler_lambda)

train_data_loader = get_data_loader(batch_size=tcfg.batch_size, split='train')
test_data_loader = get_data_loader(batch_size=tcfg.batch_size, split='validation')

Found cached dataset json (/Users/tunadorable/.cache/huggingface/datasets/noanabeshima___json/noanabeshima--TinyStoriesV2-40971520ba3bacdf/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)
Found cached dataset json (/Users/tunadorable/.cache/huggingface/datasets/noanabeshima___json/noanabeshima--TinyStoriesV2-40971520ba3bacdf/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


In [4]:
if False: # set to true if you'd like to see a graph of the learning rate schedule
    import matplotlib.pyplot as plt
    
    # Generate learning rate values
    lrs = [scheduler_lambda(i) for i in range(tcfg.max_iters)]
    
    # Plot the learning rates
    plt.figure(figsize=(10, 5))
    plt.plot(lrs, label='Learning Rate')
    plt.title('Learning Rate Schedule')
    plt.xlabel('Iteration')
    plt.ylabel('Learning Rate')
    plt.grid(True)
    plt.legend()
    plt.show()

In [5]:
model, optimizer, log_data = train(
    model, 
    tokenizer, 
    cfg, 
    optimizer,
    scheduler,
    tcfg, 
    train_data_loader,
    test_data_loader,
    #log_data: list = None, 
    #detect_anomoly = False # use if you're getting crazy errors about a the gradient being broken
)

 10%|███▊                                  | 1/10 [00:00<00:05,  1.55it/s]

step 0000: lr 0.010000, train loss 6.3184, val loss 6.3036, ppl 547, grad norm 0.4612, time elapsed: 0.50 seconds


 30%|███████████▍                          | 3/10 [00:01<00:03,  2.01it/s]

step 0002: lr 0.009831, train loss 5.8933, val loss 5.8946, ppl 363, grad norm 0.6564, time elapsed: 1.41 seconds


 50%|███████████████████                   | 5/10 [00:02<00:02,  2.09it/s]

...

step 0006: lr 0.006944, train loss 5.4089, val loss 5.4268, ppl 227, grad norm 0.5944, time elapsed: 3.20 seconds


 90%|██████████████████████████████████▏   | 9/10 [00:04<00:00,  2.17it/s]

step 0008: lr 0.001123, train loss 5.3908, val loss 5.2930, ppl 199, grad norm 0.5938, time elapsed: 4.08 seconds


100%|█████████████████████████████████████| 10/10 [00:04<00:00,  2.12it/s]

step 0009: lr 0.000100, train loss 5.3730, val loss 5.3161, ppl 204, grad norm 0.5716, time elapsed: 4.60 seconds


# inference test before you decide to save it

In [6]:
from inference import generate
prompt = "Once upon a time"
model.eval()
output = generate(
    prompt, 
    model, 
    tokenizer,
    #temperature = 0.9,
    #top_k = 32,
    #top_p = 0.9,
    #max_gen_len = 512,
    #memory_saver_div = 4,
)
model.train()
print(output[0])

Once upon a time Tom namedak mt upon Mrbk alurround wasur Lily th shPaa, th sm happyurs wasor frLTep oJ hysfke liOncee st aus was in frve toenw time analicil


# Saving your final model
if `tcfg.checkpoint_interval != None` then checkpoints have already been saved

you DO still need to do this even if you had been saving checkpoints; the final state has not yet been saved

In [7]:
from tools import save_model
save_model(model, cfg, tcfg, log_data)